In [1]:
import pandas as pd

# Load assembled METAR station data from predefined station list (united-weather-combine.py)

In [2]:
metar = pd.read_csv('data/united/united-metar-data.csv', dtype={0: str})

In [3]:
metar['DATE'] = pd.to_datetime(metar['DATE'], utc='True')

metar['STATION'] = metar['STATION'].str.zfill(11)

metar = metar.sort_values(by='DATE')

In [4]:
metar = pd.concat([metar, metar.VIS.str.split(',', expand=True)], axis=1)
metar = metar.drop([1, 2, 3], axis=1)
metar = metar.rename(
    columns={
        0: 'VIS_METERS'})
metar.VIS_METERS = pd.to_numeric(metar.VIS_METERS)

In [5]:
metar = metar.drop(columns=['WND', 'CIG', 'VIS', 'TMP', 'DEW', 'SLP'])

In [6]:
metar.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4388321 entries, 530059 to 3809503
Data columns (total 5 columns):
 #   Column       Dtype              
---  ------       -----              
 0   STATION      object             
 1   DATE         datetime64[ns, UTC]
 2   REPORT_TYPE  object             
 3   REM          object             
 4   VIS_METERS   int64              
dtypes: datetime64[ns, UTC](1), int64(1), object(3)
memory usage: 200.9+ MB


# load reference table - airport codes & stationid

In [7]:
stations = pd.read_csv('data/united/station-list.csv', header=None, dtype={0: str, 1: str}, names=['AIRPORT', 'STATION'])

In [8]:
stations['STATION'] = stations['STATION'].str.zfill(11)

In [9]:
stations.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 184 entries, 0 to 183
Data columns (total 2 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   AIRPORT  184 non-null    object
 1   STATION  184 non-null    object
dtypes: object(2)
memory usage: 3.0+ KB


# load united flight data

In [10]:
flights = pd.read_csv('data/united/united-flights.csv')

In [11]:
flights['DEST'] = flights['DEST'].str.replace(' ', '')

flights['ORIGIN'] = flights['ORIGIN'].str.replace(' ', '')

flights = flights.rename(columns={'Unnamed: 0': 'INDEX'})

In [12]:
flights['DESTOBSERVATIONTIME'] = pd.to_datetime(flights['DESTOBSERVATIONTIME'], utc='True', errors='coerce')

flights['ORIGINOBSERVATIONTIME'] = pd.to_datetime(flights['ORIGINOBSERVATIONTIME'], utc='True', errors='coerce')

flights['DEPARTUREDATE_ZULU'] = pd.to_datetime(flights['DEPARTUREDATE_ZULU'], utc='True', errors='coerce')

flights['ARRIVALDATE_ZULU'] = pd.to_datetime(flights['ARRIVALDATE_ZULU'], utc='True', errors='coerce')

flights['OUT_ZULU'] = pd.to_datetime(flights['OUT_ZULU'], utc='True', errors='coerce')

flights['IN_ZULU'] = pd.to_datetime(flights['IN_ZULU'], utc='True', errors='coerce')

In [13]:
flights.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5769 entries, 0 to 5768
Data columns (total 11 columns):
 #   Column                 Non-Null Count  Dtype              
---  ------                 --------------  -----              
 0   INDEX                  5769 non-null   int64              
 1   FLIGHTDATE             5769 non-null   int64              
 2   FLIGHTNO               5769 non-null   int64              
 3   ORIGIN                 5769 non-null   object             
 4   DEST                   5769 non-null   object             
 5   DEPARTUREDATE_ZULU     5769 non-null   datetime64[ns, UTC]
 6   ARRIVALDATE_ZULU       5769 non-null   datetime64[ns, UTC]
 7   ORIGINOBSERVATIONTIME  5593 non-null   datetime64[ns, UTC]
 8   DESTOBSERVATIONTIME    5769 non-null   datetime64[ns, UTC]
 9   OUT_ZULU               5764 non-null   datetime64[ns, UTC]
 10  IN_ZULU                5745 non-null   datetime64[ns, UTC]
dtypes: datetime64[ns, UTC](6), int64(3), object(2)
memory us

# combine flight data and weather stationid

In [14]:
flights = pd.merge(flights, stations, how='left', left_on=['DEST'], right_on=['AIRPORT'])

In [15]:
flights = flights.rename(
    columns={
        'STATION': 'DEST_STATION'})

flights = flights.drop(columns=['AIRPORT'])

flights = pd.merge(flights, stations, how='left', left_on=['ORIGIN'], right_on=['AIRPORT'])

flights = flights.drop(columns=['AIRPORT'])

flights = flights.rename(
    columns={
        'STATION': 'ORIGIN_STATION'})

In [16]:
flights.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5769 entries, 0 to 5768
Data columns (total 13 columns):
 #   Column                 Non-Null Count  Dtype              
---  ------                 --------------  -----              
 0   INDEX                  5769 non-null   int64              
 1   FLIGHTDATE             5769 non-null   int64              
 2   FLIGHTNO               5769 non-null   int64              
 3   ORIGIN                 5769 non-null   object             
 4   DEST                   5769 non-null   object             
 5   DEPARTUREDATE_ZULU     5769 non-null   datetime64[ns, UTC]
 6   ARRIVALDATE_ZULU       5769 non-null   datetime64[ns, UTC]
 7   ORIGINOBSERVATIONTIME  5593 non-null   datetime64[ns, UTC]
 8   DESTOBSERVATIONTIME    5769 non-null   datetime64[ns, UTC]
 9   OUT_ZULU               5764 non-null   datetime64[ns, UTC]
 10  IN_ZULU                5745 non-null   datetime64[ns, UTC]
 11  DEST_STATION           5769 non-null   object           

## create separate df for each date column

In [17]:
ddz = flights[['INDEX', 'FLIGHTDATE', 'FLIGHTNO', 'ORIGIN', 'ORIGIN_STATION', 'DEPARTUREDATE_ZULU']]
#ddz.info()

In [18]:
ddz = ddz.dropna(subset=['DEPARTUREDATE_ZULU'])
ddz = ddz.sort_values(by='DEPARTUREDATE_ZULU')
#ddz.shape

In [19]:
adz = flights[['INDEX', 'FLIGHTDATE', 'FLIGHTNO', 'DEST', 'DEST_STATION', 'ARRIVALDATE_ZULU']]
#adz.info()

In [20]:
adz = adz.dropna(subset='ARRIVALDATE_ZULU')
adz = adz.sort_values(by='ARRIVALDATE_ZULU')
#adz.shape

In [21]:
oot = flights[['INDEX', 'FLIGHTDATE', 'FLIGHTNO', 'ORIGIN', 'ORIGIN_STATION', 'ORIGINOBSERVATIONTIME']]
#oot.info()

In [22]:
oot = oot.dropna(subset='ORIGINOBSERVATIONTIME')
oot = oot.sort_values(by='ORIGINOBSERVATIONTIME')
#oot.shape

In [23]:
dot = flights[['INDEX', 'FLIGHTDATE', 'FLIGHTNO', 'DEST', 'DEST_STATION', 'DESTOBSERVATIONTIME']]
#dot.info()

In [24]:
dot = dot.dropna(subset='DESTOBSERVATIONTIME')
dot = dot.sort_values(by='DESTOBSERVATIONTIME')
#dot.shape

In [25]:
otz = flights[['INDEX', 'FLIGHTDATE', 'FLIGHTNO', 'ORIGIN', 'ORIGIN_STATION', 'OUT_ZULU']]
#otz.info()

In [26]:
otz = otz.dropna(subset='OUT_ZULU')
otz = otz.sort_values(by='OUT_ZULU')
#otz.shape

In [27]:
inz = flights[['INDEX', 'FLIGHTDATE', 'FLIGHTNO', 'DEST', 'DEST_STATION', 'IN_ZULU']]
#inz.info()

In [28]:
inz = inz.dropna(subset='IN_ZULU')
inz = inz.sort_values(by='IN_ZULU')
#inz.shape

## merge METAR data by 'DESTOBSERVATIONTIME'

In [29]:
dot1 = pd.merge_asof(
    left=dot, 
    right=metar, 
    left_on='DESTOBSERVATIONTIME', 
    right_on='DATE', 
    direction='nearest', 
    left_by='DEST_STATION', 
    right_by='STATION')

dot1 = dot1.rename(
    columns={
        'REM': 'DEST_UA_OBS'})

## merge METAR data by 'ORIGINOBSERVATIONTIME'

In [30]:
oot1 = pd.merge_asof(
    left=oot, 
    right=metar, 
    left_on='ORIGINOBSERVATIONTIME', 
    right_on='DATE', 
    direction='nearest', 
    left_by='ORIGIN_STATION', 
    right_by='STATION')

oot1 = oot1.rename(
    columns={
        'REM': 'ORIGIN_UA_OBS'})

## merge METAR data by 'DEPARTUREDATE_ZULU'

In [31]:
ddz1 = pd.merge_asof(
    left=ddz, 
    right=metar, 
    left_on='DEPARTUREDATE_ZULU', 
    right_on='DATE', 
    direction='nearest', 
    left_by='ORIGIN_STATION', 
    right_by='STATION')

ddz1 = ddz1.rename(
    columns={
        'REM': 'SCH_DEPARTURE'})

## merge METAR data by 'ARRIVALDATE_ZULU'

In [32]:
adz1 = pd.merge_asof(
    left=adz, 
    right=metar, 
    left_on='ARRIVALDATE_ZULU', 
    right_on='DATE', 
    direction='nearest', 
    left_by='DEST_STATION', 
    right_by='STATION')

adz1 = adz1.rename(
    columns={
        'REM': 'SCH_ARRIVAL'})

## merge METAR data by 'OUT_ZULU'

In [33]:
otz1 = pd.merge_asof(
    left=otz, 
    right=metar, 
    left_on='OUT_ZULU', 
    right_on='DATE', 
    direction='nearest', 
    left_by='ORIGIN_STATION', 
    right_by='STATION')

otz1 = otz1.rename(
    columns={
        'REM': 'ACT_DEPARTURE'})
otz1.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5764 entries, 0 to 5763
Data columns (total 11 columns):
 #   Column          Non-Null Count  Dtype              
---  ------          --------------  -----              
 0   INDEX           5764 non-null   int64              
 1   FLIGHTDATE      5764 non-null   int64              
 2   FLIGHTNO        5764 non-null   int64              
 3   ORIGIN          5764 non-null   object             
 4   ORIGIN_STATION  5764 non-null   object             
 5   OUT_ZULU        5764 non-null   datetime64[ns, UTC]
 6   STATION         5741 non-null   object             
 7   DATE            5741 non-null   datetime64[ns, UTC]
 8   REPORT_TYPE     5741 non-null   object             
 9   ACT_DEPARTURE   5741 non-null   object             
 10  VIS_METERS      5741 non-null   float64            
dtypes: datetime64[ns, UTC](2), float64(1), int64(3), object(5)
memory usage: 540.4+ KB


## merge METAR data by 'IN_ZULU'

In [34]:
inz1 = pd.merge_asof(
    left=inz, 
    right=metar, 
    left_on='IN_ZULU', 
    right_on='DATE', 
    direction='nearest', 
    left_by='DEST_STATION', 
    right_by='STATION')

inz1 = inz1.rename(
    columns={
        'REM': 'ACT_ARRIVAL'})

In [35]:
dot1.to_csv('data/united/united-flights-metar-dot.csv', index=False) # DEST OBSERVATION TIME

oot1.to_csv('data/united/united-flights-metar-oot.csv', index=False) # ORIGIN OBSERVATION TIME

ddz1.to_csv('data/united/united-flights-metar-ddz.csv', index=False) # DEPARTURE (ORIGIN) DATE ZULU

adz1.to_csv('data/united/united-flights-metar-adz.csv', index=False) # ARRIVAL (DEST) DATE ZULU

otz1.to_csv('data/united/united-flights-metar-otz.csv', index=False) # OUT ZULU

inz1.to_csv('data/united/united-flights-metar-inz.csv', index=False) # IN ZULU